In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [2]:
inpatient_specialty = pd.read_csv('/data/public/MLA/VCHAMPS-Test/inpatient_specialty_test.csv').iloc[:,1:]

In [3]:
inpatient_specialty['Specialty start date'] = inpatient_specialty['Specialty start date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['Specialty end date'] = inpatient_specialty['Specialty end date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['period'] = preprocess_data.get_period(inpatient_specialty['Specialty start date'],inpatient_specialty['Specialty end date'])

In [4]:
specialty_class = preprocess_data.generate_dict_from_csv('/home/vivi/FDA_datasets/specialty_class.csv','Specialty','Class')

In [5]:
inpatient_specialty['Specialty encoded'] = preprocess_data.preprocess_substitute_categories(specialty_class, 
                                                                               inpatient_specialty['Specialty'])

### Merge with inpatient data
Filter out specialty time before last discharge date

In [6]:
path = '/home/hassan/lily/MLA/FDA/outpatient_mortality_test.csv'
inpatient = pd.read_csv(path).iloc[:,1:]
inpatient.head()

,Internalpatientid,num_visits,CVD,last_visit_date,Age 00-20,Age 20-40,Age 40-60,Age 60-80,Age 80-100,Age 100-120,age_mean,age_std,freq,Died,away_from_death,died_within_125days
0,6,820,0,2014-10-16 01:23:22,0,0,0,356,464,0,81.400794,3.696306,58.57,1,78.0,1
1,7,6371,0,2020-11-23 13:17:39,0,0,2905,3466,0,0,60.958170,5.448019,277.00,0,NaN,0
2,9,83,0,2000-04-13 19:47:46,0,0,83,0,0,0,51.288521,0.454438,41.50,1,12.0,1
3,12,204,0,2004-07-18 11:56:33,0,0,0,204,0,0,71.991974,1.491836,34.00,1,8.0,1
4,17,252,0,2004-07-15 07:53:06,0,0,0,58,194,0,80.846346,0.903094,63.00,1,0.0,1


In [7]:
inpatient_date = inpatient[['Internalpatientid','last_visit_date']]
inpatient_date = inpatient_date.drop_duplicates(keep = 'first')
inpatient_date

,Internalpatientid,last_visit_date
0,6,2014-10-16 01:23:22
1,7,2020-11-23 13:17:39
2,9,2000-04-13 19:47:46
3,12,2004-07-18 11:56:33
4,17,2004-07-15 07:53:06
...,...,...
34791,169037,2017-10-24 20:00:40
34792,169045,2022-11-29 12:03:25
34793,169058,2003-08-17 04:14:48
34794,169059,2014-05-29 03:39:32


In [8]:
inpatient_specialty2 = inpatient_specialty.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
inpatient_specialty3 = inpatient_specialty2.loc[inpatient_specialty2['Specialty end date'] <= inpatient_specialty2['last_visit_date']]
inpatient_specialty3

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State,period,Specialty encoded,last_visit_date
0,100004,81.261813,2014-04-06 04:37:21,2014-04-08 06:17:29,CARDIAC-STEP DOWN UNIT,Florida,2 days 01:40:08,0.0,2015-04-22 09:33:31
1,100021,61.126677,2011-03-29 19:12:46,2011-03-31 21:53:53,PTSD RESIDENTIAL REHAB PROG,Colorado,2 days 02:41:07,1.0,2019-05-15 17:18:22
2,100035,52.316254,2014-07-13 14:10:34,2014-07-15 16:10:05,BLIND REHAB OBSERVATION,District of Columbia,2 days 01:59:31,1.0,2018-04-17 05:46:50
3,100041,89.583528,2014-11-05 18:42:42,2014-11-06 04:08:18,NH SHORT-STAY CONTINUING CARE,Minnesota,0 days 09:25:36,1.0,2022-12-31 13:44:12
4,100041,89.595286,2014-11-10 01:51:04,2014-11-10 01:51:04,ORTHOPEDIC,Minnesota,0 days 00:00:00,0.0,2022-12-31 13:44:12
...,...,...,...,...,...,...,...,...,...
386891,9996,77.010416,2005-12-21 07:22:13,2005-12-22 03:51:27,GEM REHABILITATION MEDICINE,Pennsylvania,0 days 20:29:14,1.0,2013-04-09 20:10:45
386892,9996,84.296993,2013-04-06 13:13:12,2013-04-08 17:12:56,GEM INTERMEDIATE CARE,Pennsylvania,2 days 03:59:44,0.0,2013-04-09 20:10:45
386893,99964,73.451708,2016-07-30 14:22:55,2016-07-30 14:22:55,GENERAL(ACUTE MEDICINE),Oregon,0 days 00:00:00,0.0,2023-06-04 13:42:57
386894,99989,71.251809,2003-10-03 12:52:03,2003-10-04 04:18:52,DOD BEDS IN VA FACILITY,Ohio,0 days 15:26:49,1.0,2011-10-26 02:41:43


In [9]:
inpatient_specialty3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375992 entries, 0 to 386895
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype          
---  ------                --------------   -----          
 0   Internalpatientid     375992 non-null  int64          
 1   Age at specialty      375992 non-null  float64        
 2   Specialty start date  375992 non-null  datetime64[ns] 
 3   Specialty end date    375992 non-null  datetime64[ns] 
 4   Specialty             375992 non-null  object         
 5   State                 375992 non-null  object         
 6   period                375992 non-null  timedelta64[ns]
 7   Specialty encoded     361290 non-null  float64        
 8   last_visit_date       375992 non-null  object         
dtypes: datetime64[ns](2), float64(2), int64(1), object(3), timedelta64[ns](1)
memory usage: 28.7+ MB


In [10]:
inpatient_specialty3['specialty count'] = inpatient_specialty3['Specialty']\
    .groupby(inpatient_specialty3['Internalpatientid']).transform('count')

specialty_medical_count = inpatient_specialty3.groupby('Internalpatientid')['Specialty encoded']\
    .apply(lambda x: (x == 0).sum()).reset_index(name='specialty medical count')

specialty_support_count = inpatient_specialty3.groupby('Internalpatientid')['Specialty encoded']\
    .apply(lambda x: (x == 1).sum()).reset_index(name='specialty support count')

inpatient_specialty3['mean age at specailty'] = inpatient_specialty3['Age at specialty']\
    .groupby(inpatient_specialty3['Internalpatientid']).transform('mean')

/tmp/ipykernel_290288/3919990082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_specialty3['specialty count'] = inpatient_specialty3['Specialty']\


/tmp/ipykernel_290288/3919990082.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_specialty3['mean age at specailty'] = inpatient_specialty3['Age at specialty']\


In [11]:
period_mean = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))

period_std = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_std['period'] = period_std['period'].apply(lambda x: np.nanstd(x.total_seconds()/86400,ddof=0))

period_mean = period_mean.rename(columns={'period': 'period mean'})
period_std = period_std.rename(columns={'period': 'period std'})

/tmp/ipykernel_290288/1693156475.py:2: RuntimeWarning: Mean of empty slice
  period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [12]:
inpatient_specialty4 = inpatient_specialty3[['Internalpatientid', 'mean age at specailty','specialty count']].sort_values('Internalpatientid')
inpatient_specialty4 = inpatient_specialty4.groupby('Internalpatientid').apply(lambda x:x)
inpatient_specialty4 = inpatient_specialty4.drop_duplicates().reset_index(drop = True)
inpatient_specialty4

/tmp/ipykernel_290288/554132651.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  inpatient_specialty4 = inpatient_specialty4.groupby('Internalpatientid').apply(lambda x:x)


,Internalpatientid,mean age at specailty,specialty count
0,7,62.879044,13
1,9,51.727595,8
2,12,72.448082,12
3,17,80.891070,29
4,22,58.983378,9
...,...,...,...
21965,168995,68.384928,7
21966,169011,64.960772,4
21967,169037,84.503238,6
21968,169059,79.918093,2


In [13]:
inpatient_specialty5 = inpatient_specialty4.merge(period_mean, how = 'left', on = 'Internalpatientid')
inpatient_specialty5 = inpatient_specialty5.merge(period_std, how = 'left', on = 'Internalpatientid')

inpatient_specialty5 = inpatient_specialty5.merge(specialty_medical_count, how = 'left', on = 'Internalpatientid')
inpatient_specialty5 = inpatient_specialty5.merge(specialty_support_count, how = 'left', on = 'Internalpatientid')

inpatient_specialty5 = inpatient_specialty5.dropna(subset = 'period mean')
inpatient_specialty5

,Internalpatientid,mean age at specailty,specialty count,period mean,period std,specialty medical count,specialty support count
0,7,62.879044,13,3.579801,7.603465,9,4
1,9,51.727595,8,1.945340,2.069379,6,2
2,12,72.448082,12,2.031376,2.952513,12,0
3,17,80.891070,29,1.622804,2.126398,12,17
4,22,58.983378,9,1.466395,2.058525,3,6
...,...,...,...,...,...,...,...
21965,168995,68.384928,7,0.668522,0.750610,4,3
21966,169011,64.960772,4,1.668035,2.455277,2,1
21967,169037,84.503238,6,11.138164,19.410162,4,2
21968,169059,79.918093,2,4.455550,4.455550,1,1


In [14]:
inpatient_specialty5.to_csv('/home/vivi/FDA_datasets/inpatient_specialty_allcause_mortality_test.csv')